Amazon Neptune
---
- graph database
- graph analytics engine
- graph machine learning + libraries & tools to store and analyze highly connected datasets

**usage (relevant to me)**
- perform vss(vector similarity search) on dense representations provided by embeddings 
and combine these results with contextually aware data representations of graphs.

**tools**
- Neptune Notebooks - do normal jupyter stuff + load and interact with data & graph
- Neptune Graph Explorer
- Neptune Developer tools
- Neptune Connectors

**cell 'magic' extensions**
- `%%opencypher` or `%%oc`: execute an opencypher query against db
- `%%graph_notebook_config`: sets nb's db configuration to input JSON payload
- `%%neptune_ml`: sets of commands to integrate with NeptuneML

**line 'magic' extensions**
- `%opencypher_status` or `%oc_status`: status of opencypher queries
- `%load`: generate a form to submit a bulk loader job
- `%status`: health status of configured host endpoint
- `%seed`: provides a form to add data to graph without the use of a bulk loader.

## check magic extensions

In [2]:
# note: must use python3 kernel only
%graph_notebook_version

4.1.0


In [3]:
%graph_notebook_config

{
  "host": "agi-research.cluster-c8uq88okovco.us-east-1.neptune.amazonaws.com",
  "neptune_service": "neptune-db",
  "port": 8182,
  "proxy_host": "",
  "proxy_port": 8182,
  "auth_mode": "DEFAULT",
  "load_from_s3_arn": "",
  "ssl": true,
  "ssl_verify": true,
  "aws_region": "us-east-1",
  "sparql": {
    "path": "sparql"
  },
  "gremlin": {
    "traversal_source": "g",
    "username": "",
    "password": "",
    "message_serializer": "graphsonv3"
  },
  "neo4j": {
    "username": "neo4j",
    "password": "password",
    "auth": true,
    "database": null
  }
}


In [4]:
%status

{'status': 'healthy',
 'startTime': 'Thu Mar 28 04:43:33 UTC 2024',
 'dbEngineVersion': '1.3.1.0.R1',
 'role': 'writer',
 'dfeQueryEngine': 'viaQueryHint',
 'gremlin': {'version': 'tinkerpop-3.6.4'},
 'sparql': {'version': 'sparql-1.1'},
 'opencypher': {'version': 'Neptune-9.0.20190305-1.0'},
 'labMode': {'ObjectIndex': 'disabled',
  'ReadWriteConflictDetection': 'enabled'},
 'features': {'SlowQueryLogs': 'disabled',
  'ResultCache': {'status': 'disabled'},
  'IAMAuthentication': 'disabled',
  'Streams': 'disabled',
  'AuditLog': 'disabled'},
 'settings': {'clusterQueryTimeoutInMs': '120000',
  'SlowQueryLogsThreshold': '5000'},
 'serverlessConfiguration': {'minCapacity': '1.0', 'maxCapacity': '20.0'}}

## learning openCypher

### 1. basic-read-queries **paradigms**:
- find
- filter
- format

In [5]:
# load data - restaurant recommendation application
%seed --model Property_Graph --dataset dining_by_friends --run

Dropdown(description='Source type:', options=('', 'samples', 'custom'), style=DescriptionStyle(description_wid…

Dropdown(description='Data model:', layout=Layout(display='none', visibility='hidden'), options=('', 'property…

Dropdown(description='Language:', layout=Layout(display='none', visibility='hidden'), options=('', 'opencypher…

Dropdown(description='Language:', layout=Layout(display='none', visibility='hidden'), options=('', 'opencypher…

Dropdown(description='Data set:', layout=Layout(display='none', visibility='hidden'), options=(), style=Descri…

...

Dropdown(description='Location:', layout=Layout(display='none', visibility='hidden'), options=('Local', 'S3'),…

FileChooser(path='/home/ec2-user/SageMaker', filename='', title='', show_hidden=False, select_desc='Select', c…

Button(description='Submit', layout=Layout(visibility='hidden'), style=ButtonStyle())

Output()

Output()

**graph**
- nodes: review, restaurant, cuisine, person, state, city
- edges: wrote, about, within, serves, friends, lives

**dataset**
- users
    - their friends
- restaurants
    - city, state, cusinine# setup visualizations
- reviews
- ratings of reviews

<table>
    <tr><th colspan=3 style="text-align: center">Element (Node/Edge) Counts</th>
    </tr>
<tr><td style="width:40%">
    
|Node Label|Count|
|:--|:--|
|review|109|
|restaurant|40|
|cuisine|24|
|person|8|
|state|2|
|city|2|
    
</td>
    <td></td>
    <td style="width:40%">

|Edge Label|Count|
|:--|:--|
|wrote|218|
|about|218|
|within|84|
|serves|80|
|friends|20|
|lives|16|

</td></tr> </table>

This dataset represents a fictitious, but realistic, restaurant recommendation application that contains:

* Users, represented by `person` nodes
* Users connected to Users via `friends` edges
* Restaurants and their associated information (`city`, `state`, `cusine`)
* Reviews include the body and ratings
* Ratings of reviews (helpful/not helpful)

In [15]:
# setup visualizations

In [14]:
%%graph_notebook_vis_options
{
    "groups": {
        "person": {
            "color": "#9ac7bf"
        },
        "review": {
            "color": "#f8cecc"
        },
        "city": {
            "color": "#d5e8d4"
        },
        "state": {
            "color": "#dae8fc"
        },
        "review_ratings": {
            "color": "#e1d5e7"
        },
        "restaurant": {
            "color": "#ffe6cc"
        },
        "cusine": {
            "color": "#fff2cc"
        }
    }
}

Visualization settings successfully changed to:

{
  "groups": {
    "state": {
      "color": "#dae8fc"
    },
    "review_ratings": {
      "color": "#e1d5e7"
    },
    "review": {
      "color": "#f8cecc"
    },
    "person": {
      "color": "#9ac7bf"
    },
    "restaurant": {
      "color": "#ffe6cc"
    },
    "cusine": {
      "color": "#fff2cc"
    },
    "city": {
      "color": "#d5e8d4"
    }
  },
  "physics": {
    "simulationDuration": 1500,
    "disablePhysicsAfterInitialSimulation": false,
    "minVelocity": 0.75,
    "barnesHut": {
      "centralGravity": 0.1,
      "gravitationalConstant": -50450,
      "springLength": 95,
      "springConstant": 0.04,
      "damping": 0.09,
      "avoidOverlap": 0.1
    },
    "solver": "barnesHut",
    "enabled": true,
    "adaptiveTimestep": true,
    "stabilization": {
      "enabled": true,
      "iterations": 1
    }
  },
  "edges": {
    "color": {
      "inherit": false
    },
    "smooth": {
      "enabled": true,
      "typ

In [22]:
import json
node_labels_json = {
    "person":"first_name",
    "city":"name",
    "state":"name",
    "restaurant":"name",
    "cusine":"name"
}
node_labels = json.dumps(node_labels_json)
node_labels

'{"person": "first_name", "city": "name", "state": "name", "restaurant": "name", "cusine": "name"}'

In [24]:
node_labels = '{"person":"first_name","city":"name","state":"name","restaurant":"name","cusine":"name"}'
node_labels

'{"person":"first_name","city":"name","state":"name","restaurant":"name","cusine":"name"}'

#### **find your data** - based on nodes and edges

- `MATCH`: specifies the pattern of data to look for
- `RETURN`: defines what and how the data will be returned to the user
- `LIMIT`: limit the data returned by specifying the maximum number of matching patterns


**pattern matching syntax**

|   openCypher Pattern|Description|
|:--|:--|
|`( )`|A node|
|`[ ]`|An edge|
|`-->`|Follow outgoing edges from a node|
|`<--`|Follow incoming edges from a node|
|`--`|Follow edges in either direction|
|`-[]->`|Include the outgoing edges in the query (to check a label or property for example)|
|`<-[]-`|Include the incoming edges in the query (to check a label or property for example)|
|`-[]-` |Include edges in either direction in the query|
|`-[]->( )`|The node on the other end of an outgoing edge|
|`<-[]-()`|The node on the other end of an incoming edge|

In [25]:
%%oc -d $node_labels
MATCH (n)  //find me nodes and label them 'n'
RETURN n   //return 'n'
LIMIT 10   //return only 10 results

In [27]:
%%oc -d $node_labels
MATCH ()-[r]->() //find me all node->edge->node patterns
RETURN r         //return the edge
LIMIT 10         //return only 10 results

In [40]:
%%oc -d $node_labels
MATCH ()-[]->(n)<-[]-() //find me all node->edge->node patterns
RETURN n         //return the nodes
LIMIT 10         //return only 10 results

In [41]:
%%oc -d $node_labels
MATCH p=()-[]->(n)<-[]-()
RETURN p
LIMIT 10

#### **filtering your data** - use attributes of nodes and edges

- WHERE clause with variety of operators
- filtering nodes by label 
    - use inline filters. add colon (`:`) followed by one or more label names (separated by a `|`)
    - use labels () and the WHERE clause
- filtering edge by type
    - use inline filters. add colon (`:`) followed by one or more label names (separated by a `|`) 
    - use type () and the WHERE clause
- finding by property or filter on attribute values
    - inline filter. add curly brackets containing the key/value you want to filter on (`{first_name: 'Dave'}`)
    - use WHERE clause
- filtering on existence of attribute or additional piece of topology
    - use `exists()` function

**operators**

|Type|Operators|
| ----------- | ----------- |
|General|`DISTINCT, x.y (property access)`|
|Math|`+`, `-`, `*`, `/`|
|Comparison|`=`, `>`, `<`, `<>`, `<=`, `>=`, `IS NULL`, `IS NOT NULL`|
|Boolean|`AND`, `OR`, `NOT`, `XOR`|
|String|`STARTS WITH`, `ENDS WITH`, `CONTAINS`, `+`|
|LIST|`+`, `IN`, `[]`|


In [43]:
%%oc -d $node_labels
MATCH (n:person)
RETURN n
LIMIT 10

In [44]:
%%oc -d $node_labels
MATCH (n)
WHERE 'person' IN labels(n)
RETURN n
LIMIT 10

In [46]:
%%oc -d $node_labels
MATCH p=(:person)-[:friends]->()
RETURN p
LIMIT 10

In [47]:
%%oc -d $node_labels
MATCH p=(:person)-[r]->()
WHERE type(r)='friends'
RETURN p
LIMIT 10

In [50]:
%%oc -d $node_labels
MATCH (n)
WHERE n.first_name="Dave"
RETURN n
LIMIT 10

In [51]:
%%oc -d $node_labels
MATCH (n {first_name: "Dave"})
RETURN n
LIMIT 10

In [53]:
%%oc -d $node_labels
MATCH (n: person)
WHERE exists(n.last_name)
RETURN n
LIMIT 10

In [54]:
%%oc -d $node_labels
MATCH (d:person)
WHERE NOT exists(d.age)
RETURN d 
LIMIT 10

In [55]:
%%oc -d $node_labels
MATCH (d:person)
WHERE NOT (d)<-[:friends]-()
RETURN d 
LIMIT 10

#### **formatting results**

In [56]:
%%oc -d $node_labels
MATCH (d:person)
RETURN *
LIMIT 10

In [57]:
%%oc -d $node_labels
MATCH (d:person)
RETURN d.first_name
LIMIT 10

In [58]:
%%oc -d $node_labels
MATCH (d:person)
RETURN d.first_name, d.last_name
LIMIT 10

In [59]:
%%oc -d $node_labels
MATCH (d:person)
RETURN DISTINCT d.last_name
LIMIT 10

In [60]:
%%oc -d $node_labels
MATCH (d)
RETURN true
LIMIT 1

In [61]:
%%oc -d $node_labels
MATCH (d:person)
RETURN d.first_name + ' ' + d.last_name AS full_name
LIMIT 10

In [62]:
%%oc -d $node_labels
MATCH (d:person)
RETURN {element: d, first: d.first_name, last: d.last_name, full: d.first_name + ' ' + d.last_name} AS full_name
LIMIT 10

### 2. variable-length-path queries

VLP or variable length path patterns

In openCypher, a basic VLP query to find all nodes within 1 to 3 hops looks like:

```
MATCH p=(:person)-[:friends*1..3]->(:person)
RETURN p
```

**Variable Length Path Syntax**

|   VLP Pattern|Description|
|:--|:--|
|`()-[*2]->()`|Find me a path containing 3 nodes and 2 edges|
|`()-[*2..3]->()`|Find me a path containing a minimum of 3 nodes and 2 edges and a maximum of 4 nodes and 3 relationships|
|`()-[*2..]->()`|Find me a path containing a minimum of 3 nodes and 2 edges, with no maximum|
|`()-[*..2]->()`|Find me a path containing a maximum of 3 nodes and 2 edges, with no minimum|
|`()-[*]->()`|Find me a path with no minimum or maximum|


Now that we have a basic understanding of openCypher's VLP syntax, let's look at how this is applied to answer some common graph query patterns.

**Static Length paths**

The simplest VLP pattern you can do in openCypher is to specify a fixed number of loops/iterations for your pattern.  This is accomplished using the syntax `()-[:friends*2]->()`.  Let's execute the query below to search for patterns containing 3 nodes and 2 edges.

In [74]:
%%oc -d $node_labels
MATCH p=()-[:friends*2]->()
RETURN p  
LIMIT 10 

In [70]:
%%oc -d $node_labels
MATCH p=()-[:friends*2..]->()
RETURN p
LIMIT 10

In [75]:
%%oc -d $node_labels
MATCH p=()-[:friends*..2]->()
RETURN p
LIMIT 10

In [72]:
%%oc -d $node_labels
MATCH p=()-[:friends*]->()
RETURN p  
LIMIT 100   

#### exercises
    
1. find the friends of Dave's friends using a VLP
```
%%oc -d $node_labels
MATCH p=({first_name: "Dave"})-[:friends*2]->(x)
RETURN DISTINCT x.first_name
LIMIT 10
```

2. find all person nodes connected to Dave
```
%%oc -d $node_labels
MATCH p=({first_name: "Dave"})-[:friends*]->()
RETURN p
LIMIT 10
```

3. find if dave and denise are connected
```
%%oc -d $node_labels
MATCH p=(d {first_name: "Dave"})-[:friends*]-(x {first_name: "Denise"})
RETURN true
LIMIT 1
```

4. find all ways dave and denise are connected
```
%%oc -d $node_labels
MATCH p=(d {first_name: "Dave"})-[:friends*]-(x {first_name: "Denise"})
RETURN p
LIMIT 100
```

### 3: ordering-functions-grouping

**ordering**
- ordering by a property
    - `ORDER BY <variable>.<property name>`
    - `ORDER BY <variable>.<property name> DESC`
- ordering by multiple properties
    - the results are first ordered by the first property, then for equal values, the next property, and so on for all the specified properties.  
- ordering by expressions such as `id()` or `keys()`


**functions**
|Type|Function|
| ----------- | ----------- |
|Predicate|`exists()`|
|Scalar|`coalesce()`, `endNode()`, `head()`, `id()`, `last()`, `length()`, `properties()`, `size()`, `startNode()`, `timestamp()`, `toBoolean()`, `toFloat()`, `toInteger()`, `type()`|
|Aggregating|`avg()`, `collect()`, `count()`, `max()`, `min()`, `sum()`|
|List|`keys()`, `labels()`, `nodes()`, `range()`, `relationships()`, `reverse()`, `tail()`|
|Math - numeric|`abs()`, `ceil()`, `floor()`, `rand()`, `round()`, `sign()`|
|Math - logarithmic|`e()`, `exp()`, `log()`, `log10()`, `sqrt()`|
|String|`left()`, `lTrim()`, `replace()`, `reverse()`, `right()`, `rTrim()`, `split()`, `substring()`, `toLower()`, `toString()`, `toUpper()`, `trim()`|


**grouping**
- different than SQL or gremlin
- aggregation functions (`avg()`, `collect()`, `count()`, `max()`, `min()`, `sum()`)
- for grouping to work, the final aggregtion expression has to be either:
    - an aggregation function (`RETURN count(*)`)
    - a grouping key (`RETURN n, count(n)`)
    - a local variable
- these expressions ensure that the aggregation is be computed over all the results within a group.  
    - Groups are determined through the grouping keys. 
    - Grouping keys are non-aggregate expressions, that are specfied in conjunction with the aggregate functions are are used to group the values.
- group by a property
- group on a pattern match


**combining queries**
- combine subqueries to create more complex queries
- how ?
    - `UNION`
        - combine results of 2 or more queries and return combined result. (no duplicates)
        - note: must return same number of columns and the columns must have identical names
    - `UNION ALL`
        - same as UNION but retains duplicates
    - `WITH`
        - subqueries are chained together instead of combining the results
        - usage patterns:
            - limiting the no. of entries passed to other subqueries
            - introducing new intermediate results such from projection/aggregation/etc
            - filtering on aggregated values for subsequent queries

In [76]:
%%oc -d $node_labels
MATCH (n:restaurant)  
RETURN n.name, 

In [77]:
%%oc -d $node_labels
MATCH (n:restaurant)  
RETURN n.name
ORDER BY n.name DESC

In [78]:
%%oc -d $node_labels
MATCH (n:restaurant)  
RETURN n.name, n.address
ORDER BY n.name, n.address

In [79]:
%%oc -d $node_labels
MATCH (n:restaurant)  
RETURN n.name
ORDER BY n

In [80]:
%%oc -d $node_labels
MATCH (n:restaurant)  
RETURN n.name
ORDER BY id(n)

In [81]:
%%oc -d $node_labels
MATCH (n:restaurant)  
RETURN toUpper(n.name) as name, 
    //returns the first non-null answer, since this property does not exist it should be `No Capacity Provided`
    coalesce(n.max_capacity, 'No Capacity Provided') as capacity, 
    n.address as address
ORDER BY n.name
LIMIT 10

In [82]:
%%oc -d $node_labels
MATCH (n:restaurant)  
WHERE toUpper(n.name) = 'HAND ROLL'
RETURN n.name

In [83]:
%%oc -d $node_labels
MATCH (n:restaurant)  
RETURN avg(size(split(n.name, " ")))
LIMIT 10

In [84]:
%%oc -d $node_labels
MATCH (n)
RETURN n.name AS name, count(n.name) AS cnt

In [85]:
%%oc -d $node_labels
MATCH (n:restaurant)  
RETURN count(*) AS count_in_length, size(n.name) AS name_length
ORDER BY name_length

In [86]:
%%oc -d $node_labels
MATCH (n:restaurant)<-[:about]-(r:review)
RETURN n.name AS name, avg(r.rating) AS rating
ORDER BY rating desc

In [109]:
%%oc -d $node_labels
MATCH (n:restaurant)
RETURN n
LIMIT 5
UNION
MATCH (n:review)
RETURN n
LIMIT 5

In [110]:
%%oc -d $node_labels

MATCH (n:restaurant)
WITH n LIMIT 5
MATCH p=(n)<-[:about]-(:review)
RETURN p

In [ ]:
%%oc -d $node_labels
MATCH (r:restaurant)<--(rev:review)
WITH r, toUpper(r.name) AS upperCaseName
WHERE upperCaseName STARTS WITH 'WITH'
RETURN r.name

In [113]:
%%oc -d $node_labels
MATCH (r:restaurant)<--(rev:review)
WITH size(collect(rev)) as num_reviews, r
WHERE num_reviews > 5
MATCH (r)-[:within]->(c:city)
RETURN c.name as city, r.name as name

In [112]:
%%oc -d $node_labels
MATCH (r:restaurant)<--(rev:review)
WITH r, count(rev) as num_reviews
WHERE num_reviews > 5
MATCH (r)-[:within]->(c:city)
RETURN c.name as city, r.name as name

In [117]:
%%oc -d $node_labels
MATCH (r:restaurant)<--(rev:review)
WITH size(collect(rev)) as num_reviews, r
WHERE num_reviews >5
RETURN rev

In [116]:
%%oc -d $node_labels
MATCH (r:restaurant)<--(rev:review)
WITH collect(rev) as revs
WHERE size(revs)>5
RETURN revs

Question: What are the 3 highest restaurants?

Using the data model above, write a query that will:

* Find the 3 highest average restaurant rating
* Find the associated cuisine
* Return the restaurant name, the cuisine name, and the average rating
* Order the results by average rating descending

The results for this query are:

|Restaurant name|Cuisine|Avg Rating|
|---|---|---|
|Lonely Grape|bar|5.0|
|Perryman's|bar|4.5|
|Rare Bull|steakhouse|4.333333|


In [119]:
%%oc -d $node_labels
MATCH (r:restaurant)-->(c:cuisine)
WITH r, c
MATCH (r)<--(rev:review)
RETURN r.name AS `Restaurant name`, c.name AS `Cuisine`, avg(rev.rating) AS `Avg Rating`
ORDER BY `Avg Rating` desc
LIMIT 3

In [127]:
%%oc
MATCH (r:restaurant)<-[:about]-(rev:review)
WITH r, avg(rev.rating) AS avg_rating
ORDER BY avg_rating DESC
LIMIT 3
MATCH (r)-[:serves]->(c:cuisine)
RETURN r.name, c.name, avg_rating
ORDER BY avg_rating Desc

Question: Find the top 3 highest rated restaurants in the city where Dave lives

Using the data model above, write a query that will:

* Find a `person` node(s) with a `first_name` of "Dave"
* Find the `city` that Dave lives in
* Find the average rating of restaurants in that city
* Find the top 3 average ratings
* Return the restaurant name, address, and average rating
* Order by the average rating descending

The results for this query are:

|Restaurant name|Address|Avg Rating|
|---|---|---|
|Dave's Big Deluxe|	490 Ivan Cape|4.0|
|Pick & Go|4881 Upton Falls|3.75|
|Without Chaser|	01511 Casper Fall|3.5|

In [120]:
%%oc -d $node_labels
MATCH (p {first_name: "Dave"})-->(c:city)
WITH c
MATCH (c)<--(r:restaurant)
WITH r
MATCH (r)<--(rev:review)
RETURN r.name AS `Restaurant name`, r.address AS `Address`, avg(rev.rating) AS `Avg Rating`
ORDER BY `Avg Rating` desc
LIMIT 5

In [126]:
%%oc
MATCH (p:person {first_name: 'Dave'})-[:lives]->(:city)<-[:within]-(r:restaurant)<-[:about]-(v:review)
WITH r, avg(v.rating) AS rating_average, p
RETURN r.name AS name,
r.address AS address, rating_average
ORDER BY rating_average DESC 
LIMIT 3

Question: What Mexican or Chinese restaurant near Dave that is the highest rated?

Using the data model above, write a query that will:

* Find a `person` node(s) with a `first_name` of "Dave"
* Find the `city` that Dave lives in
* Find the restaurants in that city that serve 'Mexican' or 'Chinese' food
* Find the average rating of those restaurants
* Return the restaurant name, address, and average rating
* Order by the average rating descending
* Return the top 1 result

The results for this query are:

|Restaurant name|Address|Avg Rating|
|---|---|---|
|With Salsa|24320 Williamson Causeway|3.5|

In [121]:
%%oc -d $node_labels
MATCH (p {first_name: "Dave"})-->(c:city)
WITH c
MATCH (c)<--(r:restaurant)
WITH r
MATCH (r:restaurant)-->(cu:cuisine)
WHERE cu.name="Mexican" OR cu.name="Chinese"
WITH r
MATCH (r)<--(rev:review)
RETURN r.name AS `Restaurant name`, r.address AS `Address`, avg(rev.rating) AS `Avg Rating`
ORDER BY `Avg Rating` desc
LIMIT 1

In [125]:
%%oc
MATCH (p:person {first_name: 'Dave'})-[:lives]->(:city)<-[:within]-(r:restaurant)-[:serves]->(c:cuisine) 
WHERE c.name IN ['Mexican', 'Chinese'] 
WITH r
MATCH (r)<-[:about]-(v:review) 
WITH r, avg(v.rating) AS rating_average 
RETURN r.name AS name, 
    r.address AS address, rating_average
ORDER BY rating_average DESC 
LIMIT 1

Question: What are the top 3 restaurants, recommended by his friends, where Dave lives? (Personalized Recommendation)

Using the data model above, write a query that will:

* Find a `person` node(s) with a `first_name` of "Dave"
* Find the `city` that Dave lives in
* Find Dave's friends
* Find reviews written by Dave's friends in the city "Dave" lives in
* Find the average rating of those restaurants
* Return the restaurant name, address, and average rating
* Order by the average rating descending
* Return the top 3

The results for this query are:

|Restaurant name|Address|Avg Rating|
|---|---|---|
|Dave's Big Deluxe|490 Ivan Cape|4.0|
|With Salsa|24320 Williamson Causeway|4.0|
|Satiated|370 Hills Estates|3.666667|

In [123]:
%%oc
MATCH (p:person {first_name: 'Dave'})-[:lives]->(c:city)
MATCH (p)-[:friends]-()-[:wrote]-(v)-[:about]-
(r:restaurant)-[:within]-(c) 
RETURN  r.name AS name, r.address AS address, avg(v.rating) as rating_average
ORDER BY rating_average DESC
LIMIT 3

### 4. create-update-delete queries

In [ ]:
%%oc -d $node_labels
CREATE (n:person {first_name: 'John', last_name: 'Doe'})
RETURN n

In [ ]:
%%oc -d $node_labels
CREATE (n:person)
SET n={first_name: 'Jane', last_name: 'Doe'}
RETURN n

In [ ]:
%%oc -d $node_labels
MATCH (john:person {first_name: 'John', last_name:'Doe'}), (jane:person {first_name: 'Jane', last_name:'Doe'})  
CREATE (john)-[r:friends]->(jane)
RETURN r

In [ ]:
%%oc -d $node_labels
MATCH (john:person {first_name: 'John', last_name:'Doe'}), (jane:person {first_name: 'Jane', last_name:'Doe'})  
CREATE (john)-[r:friends]->(jane)
SET r.relationship='coworker'
RETURN r

In [ ]:
%%oc -d $node_labels
CREATE p= (jim:person {first_name: 'Jim', last_name: 'Doe'})-[:friends]->
    (joe:person {first_name: 'Joe', last_name: 'Doe'})
RETURN p

In [ ]:
%%oc -d $node_labels
MATCH (joe:person {first_name: 'Joe', last_name: 'Doe'})  
SET joe.first_name='Joseph'
RETURN joe

In [ ]:
%%oc -d $node_labels
MATCH (joe:person {first_name: 'Joseph', last_name: 'Doe'})  
SET joe += {age: 43}
RETURN joe

In [ ]:
%%oc -d $node_labels
MATCH (joe:person {first_name: 'Joseph', last_name: 'Doe'})  
SET joe.age = null
RETURN joe

In [ ]:
%%oc -d $node_labels
MATCH (n:person {first_name: 'Steve'})
DELETE n
RETURN n

In [ ]:
%%oc
MATCH (n:person {first_name: 'Joseph'})-[r]-()
DELETE r
RETURN r

In [ ]:
%%oc -d $node_labels
MATCH (n:person {first_name: 'John', last_name: 'Doe'})
DETACH DELETE n
RETURN n